# Tugas Besar 2 - Regression

<p>Dibuat oleh:</p>

William Rukmansa - 13516066 <br>
Bella Destiana Junaidi - 13516070 <br>
Kevin Andrian Liwinata - 13516118 <br>
Nuha Adinata - 13516120

## 1. Import Data Hasil Web Crawling

In [1]:
import json
import copy
import pandas as pd
import numpy as np
import datetime as dt
from datetime import date
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

In [2]:
# Buka file JSON
file = 'car.json'
with open(file) as json_file:
    json_data = json.load(json_file)

# Konversi JSON menjadi dataframe
data = pd.DataFrame.from_records(json_data)
for i in data.columns:
    if data[i].transform(type).any() == list:
        data = data.drop(i, axis= 1)
data.head()

,Merek,Model,Varian,Tahun,Jarak tempuh,Tipe bahan bakar,Warna,Transmisi,Sistem Penggerak,Tahun_Beli_Mobil,Lokasi,Waktu_Penjualan,Nama_Penjual,Waktu_Penjual_Bergabung_ke_OLX,Dihighlight,Harga,Tipe bodi,Kapasitas mesin,Tipe Penjual,Nama Bursa Mobil
0,Daihatsu,Xenia,R 1.3 STD,2016,25.000-30.000 km,Bensin,Silver,Manual,4X2,2016,"Mulyorejo, Surabaya Kota, Jawa Timur",13 Agt,Chris,Jan 2012,True,Rp 142.500.000,NaN,NaN,NaN,NaN
1,Mitsubishi,Xpander,NaN,2019,0-5.000 km,Bensin,Lainnya,Manual,NaN,2019,"Tangerang, Tangerang Kota, Banten",9 Okt,trierd1802,Mei 2018,True,Rp 235.800.000,NaN,NaN,NaN,NaN
2,Toyota,Vellfire,G,2016,15.000-20.000 km,Bensin,Putih,Automatic Triptonic,4X2,2016,"Kemayoran, Jakarta Pusat, Jakarta D.K.I.",10 Okt,Dexi Auto PMK S40,Okt 2015,True,Rp 788.000.000,Van,>2.000 - 3.000 cc,Diler,Pasar Mobil Kemayoran
3,BMW,Serie 6,640i,2014,25.000-30.000 km,Bensin,Putih,Automatic,Rear Wheel Drive (RWD),2014,"Kebayoran Lama, Jakarta Selatan, Jakarta D.K.I.",5 hari yang lalu,Kredit Murah BCA Finance,Agt 2014,True,Rp 975.000.000,Coupe,>3.000 cc,Diler,Bursa Mobil Bintaro
4,Toyota,Avanza,G,2014,55.000-60.000 km,Bensin,Putih,Manual,NaN,2014,"Lowokwaru, Malang Kota, Jawa Timur",25 Sep,Frendy chandra,Jan 2016,True,Rp 157.000.000,NaN,>1.000 - 1.500 cc,Diler,NaN


## 2. Exploratory Data Analysis

In [3]:
data.describe()

,Merek,Model,Varian,Tahun,Jarak tempuh,Tipe bahan bakar,Warna,Transmisi,Sistem Penggerak,Tahun_Beli_Mobil,Lokasi,Waktu_Penjualan,Nama_Penjual,Waktu_Penjual_Bergabung_ke_OLX,Dihighlight,Harga,Tipe bodi,Kapasitas mesin,Tipe Penjual,Nama Bursa Mobil
count,25,25,23,25,25,25,25,25,20,25,25,25,25,25,25,25,19,21,21,6
unique,11,17,13,12,14,2,8,3,5,12,18,6,20,19,2,17,9,4,2,4
top,Toyota,Xenia,G,2014,25.000-30.000 km,Bensin,Putih,Manual,4X2,2014,"Laweyan, Surakarta Kota, Jawa Tengah",Hari ini,Chris,Agt 2014,False,Rp 120.000.000,Minibus,>1.000 - 1.500 cc,Individu,Pasar Mobil Kemayoran
freq,8,4,7,5,5,24,8,15,11,5,3,15,2,2,20,3,5,10,14,2


In [4]:
for column in data.columns:
    print(data[column].describe())
    print("\n")

count         25
unique        11
top       Toyota
freq           8
Name: Merek, dtype: object


count        25
unique       17
top       Xenia
freq          4
Name: Model, dtype: object


count     23
unique    13
top        G
freq       7
Name: Varian, dtype: object


count       25
unique      12
top       2014
freq         5
Name: Tahun, dtype: object


count                   25
unique                  14
top       25.000-30.000 km
freq                     5
Name: Jarak tempuh, dtype: object


count         25
unique         2
top       Bensin
freq          24
Name: Tipe bahan bakar, dtype: object


count        25
unique        8
top       Putih
freq          8
Name: Warna, dtype: object


count         25
unique         3
top       Manual
freq          15
Name: Transmisi, dtype: object


count      20
unique      5
top       4X2
freq       11
Name: Sistem Penggerak, dtype: object


count       25
unique      12
top       2014
freq         5
Name: Tahun_Beli_Mobil, dtype: object

## 3. Data Preprocessing

### 3.1.  Preprocessing Tanggal Waktu Penjualan

In [5]:
# Preprocessing Tanggal Waktu Penjualan
for datas in data['Waktu_Penjualan']:

    # ubah 'Hari ini'
    if(datas.find("Hari")==0):
        datas = date.today().strftime("%d %b")
        
    # ubah 'x hari yang lalu'
    if(datas.find("hari")==2):
        day = datas.split()
        d = int(day[0])
        dates = date.today()- dt.timedelta(days=d)
        datas = dates.strftime("%d %b")

### 3.2. Null/Missing Data Processing

### 3.3. Pemisahan Input dan Target Data

In [6]:
# Define input data (x)
x_data = copy.deepcopy(data)
x_data.drop('Harga', axis=1, inplace=True)

# Define target data (y)
y_data = copy.deepcopy(data['Harga'])

# Split training-test data
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2)

### 3.4. Ubah menjadi Category

In [7]:
columns = list(x_data)
x_data[columns] = x_data[columns].astype('category')

### 3.5. Scaling

### 3.6. Normalization

### 3.7. Feature Selection

## 4. Regression Model Creation

## 5. Model Evaluation